In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a list of the flights available according to what user asks for, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a list of all the flights available in a table format in markdown. The columns of the table should be - Flight carrier, Flight Dat and times, Fare, No. of stops. \
Provide exact flight carriers. If it includes ads or offers, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:
def fetch_flights(from_tx,to_tx, date_from,date_to=''):
    website = Website(f"https://www.ca.kayak.com/flights/{from_tx}-{to_tx}/{date_from}/{date_to}")
    user_prompt = user_prompt_for(website)
    messages = [{"role":"system","content":system_prompt},{"role":"user","content":user_prompt}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    display(Markdown(response.choices[0].message.content))

In [8]:
fetch_flights('yyz','del','2025-11-09')

Here is the list of available flights from YYZ to DEL on 9/11:

| Flight Carrier | Flight Date and Times | Fare   | No. of Stops |
|----------------|-----------------------|--------|--------------|
| Air Canada     | 9/11, 10:00 AM        | C$ 833 | 1            |
| Lufthansa      | 9/11, 5:00 PM         | C$ 847 | 2            |
| Qatar Airways  | 9/11, 1:30 PM         | C$ 1,559| 1            |

### Summary of Offers
- The cheapest fare is C$ 833 with a travel time of 23 hours and 35 minutes.
- The best fare option is C$ 847 with a travel time of 22 hours and 20 minutes.
- The quickest option is priced at C$ 1,559 with a travel duration of 13 hours and 55 minutes. 

*Note: Prices are per person and do not include baggage fees.*